# Toy Intraday Algo

## Basic Framework

### 0 Project meta (root)
* 0.1 `README.md`
* 0.2 `pyproject.toml`
* 0.3 Makefile
* 0.4 `.gitignore`

### 1 `config/`: Configuration
* 1.1 `config/settings.yaml`: global parameters
* 1.2 `.env.example`: API keys, etc.

### 2 `utils/`: Utilities
* 2.1 `utils/calendars.py`: market hours/holidays
* 2.2 `utils/time.py`: tz handling
* 2.3 `utils/io.py`: parquet/cache helpers
* 2.4 `utils/validation.py`: data & backtest guards
* 2.5 `utils/seeds.py`: Global RNG control for reproducibility (slippage noise, bootstraps)

### 3 `data/`: Data: csv minute bars (symbol, datetime, open, high, low, close, volume)
* 3.1 `data/schema.py`: Pydantic models (or pandera) to validate every input table (OHLCV, corporate actions)
* 3.2 `data/ingest.py`
* 3.3 Corporate actions: Split/dividend adjustment utilities

### 4 `engine/`: Engine (core): define the abstractions before strategies
* 4.1 `engine/orders.py`: Order objects & enums (MKT/LMT/STOP, time-in-force, reduce-only)
* 4.2 `engine/broker_core.py`: Paper broker (fills, slippage, fees)
* 4.3 `engine/risk.py`: Stops, daily loss limits, max leverage
* 4.4 `engine/datafeed.py`: Loads & resamples data
* 4.5 Feature store: Cache precomputed indicators per (symbol, day, params)
* 4.6 Slippage models: e.g., `engine/fixed_bps.py`, `engine/spread_atr.py`
* 4.7 Cost models: fees, SEC/TAF, borrow (if short)
* 4.8 Position sizing: e.g., `engine/vol_target.py`, `engine/notional_cap.py`
* 4.9 Portfolio: `engine/allocator.py`, `engine/exposure.py`
* 4.10 `engine/trade_log.py`: Canonical trade/position ledger (fills, fees, M2M, reasons for entry/exit)
* 4.11 `engine/metrics.py`: PnL, Sharpe, drawdown, hit rate
* 4.12 `engine/backtester.py`: Event loop

### 5 `strategies/`: Strategies
* 5.1 `strategies/base.py`: Strategy interface
#### Mean Reversion
* **5.2** `strategies/vwap_mean_reversion.py`: Go long when price is below session VWAP by a threshold (betting it reverts upward) and short when above (betting it reverts downward)
* **5.3** `strategies/bollinger_mean_reversion.py`: Fade moves when price closes outside Bollinger Bands
* 5.4 `strats/rsi_reversal.py` Go long when RSI < 30, short when RSI > 70 (intraday overbought/oversold)
* **5.5** `strategies/opening_reversal.py`: Fade extreme opening gaps or first-N-minute spikes
* **5.6** `strategies/pairs_mean_reversion.py`: Trade z-score of spread between two correlated stocks intraday
#### Momentum / Trend
* 5.7 `strategies/breakout.py`: Buy when price breaks above a recent rolling high, and short when it breaks below a rolling low (betting on momentum continuation)
* 5.8 `strategies/intraday_momentum.py`: Enter when returns over last N bars show strong directional trend
* **5.9** `strategies/moving_average_crossover.py`: Long when fast MA crosses above slow MA; short on reverse
* 5.10 `strategies/intraday_breakout_volatility.py`: Breakouts filtered by volatility expansion (ATR filter)
#### Statistical / Volatility
* 5.11 `strategies/volatility_scalping.py`: Trade volatility crush: fade IV proxy spikes (e.g., realized intraday range)
* 5.12 `strategies/momentum_vol_filter.py`: Only trade breakouts if realized vol > rolling vol threshold
#### Simpler Heuristics
* 5.13 `strategies/time_of_day_bias.py`: Capture structural effects (e.g., morning momentum, afternoon mean-reversion)
* 5.14 `strategies/overnight_gap_follow.py`: Trade intraday continuation of large overnight gaps (long if up gap, short if down)

### 6 `run_backtest.py`: Backtest (CLI entry point)

### 7 `experiments/`: Experiments
* 7.1 `experiments/grid_search.py`
* 7.2 `experiments/walk_forward.py`
* 7.3 Registry: results (CSV/JSON/SQLite of runs)

### 8 `reporting/`: Reporting
* 8.1 `reporting/tearsheet.py`: summary figs & HTML report
* 8.2 `reporting/plots.py`

### 9 `adapters/`: Adapters (optional until paper/live)
* 9.1 Data: vendor-specific loaders
* 9.2 Broker: paper/live API clients

### 10 `live_runner/`: Live runner (optional)
* 10.1 `live_runner/run_live.py`: same interface as backtester

### 11 `tests/`: Tests
* 11.1 `tests/test_datafeed.py`
* 11.2 `tests/test_broker.py`
* 11.3 `tests/test_metrics.py`
* 11.4 `tests/test_strategies.py`

### 12 `notebooks/`: Notebooks
* 12.1 `note/explore.ipynb`: quick EDA & parameter sweeps

### 13 `artifacts/`: Artifacts
* 13.1 State: run outputs, checkpoints
* 13.2 Logs